# Course Timetabling: Estudo de caso para o problema de agendamento de grade horária

## Fase 1: Alocação de professor em disciplina

Objetivo:
- Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) e o número de restrições fracas atendidas
- Alocar professores efetivos e substitutos em disciplinas (obrigatórias, eletivas e de serviço) conforme as regras do instituto da computação da UFRJ

### Modelagem

Conjuntos:

- $P$: Conjunto de professsores $p \in P$
- $PP$: Subconjunto de professores permanentes $PP \in P$
- $PS$: Subconjunto de professores substitutos $PS \in P$
- $D$: Conjunto de disciplinas ofertadas atrelada a uma turma T com o detalhamento da disciplina $d \in D$
- $Cred_d$: Conjunto de créditos das disciplinas $Cred_d \in D$
- $DAptas_p \subset D$: Subconjunto de disciplinas nas quais o professor p está habilitado a dar aulas.
- $H_d$: Subconjunto de horários das disciplinas $H_d \in D$
- $DS_d$: Subconjunto de dias da semana das disciplinas $DS_d \in D$
- $H_d$: Subconjunto com todos os horários possíveis para uma disciplina $H_d \in D$
- $DS_d$: Subconjunto com todos os dias da semana possíveis para uma disciplina $DS_d \in D$
- $D_h$: Subconjunto de disciplinas que possuem o horário h ($h \in H_d$)
- $D_{ds}$: Subconjunto de disciplinas que possuem o dia da semana ds ($ds \in DS_d$)



<!-- - $D_p \subset D$: Preferências de disciplinas de cada professor p -->
<!-- - $Tp \subset H Preferência de turno de cada professor p; -->

#### Variáveis

1) Alocação de professor em disciplina de uma turma 


$$
X_{p,d} \in (0,1) =

\begin{cases}
1 &  \quad \text{se o professor p for alocado na disciplina d}\\ 
0 & \quad \text{caso contrário} 
\end{cases}

$$

$$
p \in P, d \in D
$$

2. Variável de folga (slack variable) que indica quantos créditos o professor permanente está abaixo do ideal pela coordenação

$$
PNC_p \in \Z \quad p \in PP
$$ 


#### Coeficientes


1) Professor habilitado em dar disciplina recebe coeficiente 1

$$

f_{p,d} \in (0,1) = 

\begin{cases}
1 &  \quad d \in DAptas_p\\ 
0 & \quad \text{caso contrário}
\end{cases}

$$

$$
p \in P, d \in D \wedge	p \neq \text{'DUMMY'}
$$

2) Professor DUMMY é habilitado em todas as disciplinas. Ele recebe coeficiente 0.0001 pois só deve ser considerado se não existir outro professor apto.

$$

f_{p,d} = 0.0001 \quad d \in D, p \in P \wedge p = \text{'DUMMY'}

$$

3) Fator de penalidade no caso do professor não atingir a quantidade de créditos mínima

$$
    Wf = 1000
$$



### Função objetivo

Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) 
<!-- e o número de restrições fracas atendidas -->

$$
MAX \sum_{p \in P}\sum_{d \in D} f_{p,d}X_{p,d} - W_f \sum_{p \in PP} PNC_p
$$

### Restrições

### Restrições fracas

RF1: Garante que o professor seja alocado com a quantidade de créditos sujerida pela coordenação se possível. Não inviabilisa o modelo caso não seja atingido. 

Considera: 
- $FAL_{pp}$: Valor fixo que representa a quantidade de créditos que um professor deve ministrar. Número de créditos mínimos: 8. Cada professor precisa dar no mínimo 8 créditos horas de aula por semana (cada disciplina geralmente possui 4 créditos).


$$
\sum_{d \in D} cred_d X_{p,d} = FAL_{pp} - PNC_p \quad p \in PP \wedge	p \neq \text{'DUMMY'}
$$

##### Restrições fortes

RH2: Regime de trabalho (quantidade de horas) - quantidade de créditos máximo para o professor substituto

- $FAL_{ps}$: Número de créditos máximos: 12. Às vezes professores podem dar 12 créditos por semana. Para a equação, temos o professor p fixado e pertencente ao conjunto PS.

$$
\sum_{d \in D} cred_d X_{p,d} \leq FAL_{ps} \quad p \in PS \wedge	p \neq \text{'DUMMY'}
$$

RH3: Uma disciplina de uma turma, deverá ser ministrada por um único professor

$$
\sum_{p \in P} X_{p,d} = 1  \quad d \in D
$$

RH4: Um professor poderá dar no máximo 1 disciplina de uma turma em um mesmo dia e horário (binário OU >= 1)


$$
\sum_{d \in (D_h \cap D_{ds})} X_{p,d} \leq 1  \quad p \in P, h \in H_d, ds \in DS_d \wedge	p \neq \text{'DUMMY'}
$$

RH5: Um professor não pode lecionar uma disciplina em que ele não esteja apto

$$
\sum_{d \in (\Omega - DAptas_d)} X_{p,d} = 0 \quad p \in P \quad DAptas_d \in D
$$





In [5]:
# https://developers.google.com/sheets/api/quickstart/python

import os.path
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

SAMPLE_SPREADSHEET_ID = "1MZ_LFco9SZ5FrZFt7CAUCJu4xHov9VtTuil0CyRx9jI"



def read_google_sheet_to_dataframe(spreadsheet_id, range_name):
    """Reads data from a Google Sheet and returns it as a pandas DataFrame."""
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "../credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)

        sheet = service.spreadsheets()
        result = (
            sheet.values()
            .get(spreadsheetId=spreadsheet_id, range=range_name)
            .execute()
        )
        values = result.get("values", [])

        if not values:
            print("No data found.")
            return pd.DataFrame()

        # Convert the values to a pandas DataFrame
        df = pd.DataFrame(values[1:], columns=values[0])
        return df

    except HttpError as err:
        print(err)
        return pd.DataFrame()
        

In [6]:
SAMPLE_RANGE_NAME = "professores!A:K"
df = read_google_sheet_to_dataframe(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME)
print(df)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=829831545956-elqljtporpi0kvhml2bqelfdpeij0vbs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52186%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=2rbr67I3WPklzcXAfCOOTr3uEYC4kc&access_type=offline
   Alocar                        Nome curto     Disciplinas aptas  \
0    TRUE                  Adriana Vivacqua         ICP145,ICP616   
1   FALSE  Adriano Joaquim de Oliveira Cruz                         
2   FALSE                      Ageu Pacheco                         
3    TRUE                      Aloisio Pina                         
4    TRUE                    Amanda Camacho                         
..    ...                               ...                   ...   
70   TRUE                   Thiago Henrique                         
71   TRUE                       Thiago José                         
72   TRU

In [21]:
categorias = ["PS", "EX", "AP"]
professores_permanentes = df.loc[(df["Alocar"] == "TRUE") & (~df["Categoria"].isin(categorias))]

professores_permanentes

,Alocar,Nome curto,Disciplinas aptas,Nome completo,Categoria,Nível,Área de conhecimento,Abreviatura,SIAPE,E-mail,Posse
0,TRUE,Adriana Vivacqua,"ICP145,ICP616",Adriana Santarosa Vivacqua,PP,Associado,"ED,ES,H",ASV,1672176,avivacqua@ic.ufrj.br,25/09/2009
3,TRUE,Aloisio Pina,,Aloisio Carlos de Pina,PP,Adjunto,"CD,TC",ACP,3565609,aloisiopina@ic.ufrj.br,03/12/2013
5,TRUE,Amaury Cruz,,Amaury Alvarez Cruz,PP,Adjunto,CC,AAC,3065763,amaury@ic.ufrj.br,29/08/2018
6,TRUE,Anamaria Moreira,ICP133,Anamaria Martins Moreira,PP,Titular,"ES,TC",AMM,1258224,anamaria@ic.ufrj.br,05/03/2014
8,TRUE,Angela Gonçalves,,Angela Maria Silva Gonçalves,PP,Adjunto,CC,AMG,7366082,angela@ic.ufrj.br,15/02/1996
10,TRUE,Carla Delgado,ICP472,Carla Amor Divino Moreira Delgado,PP,Associado,"CD,TC,H",CAD,1850888,carla@ic.ufrj.br,2010
11,TRUE,Carolina Marcelino,,Carolina Gil Marcelino,PP,Adjunto,CD,CGM,1614930,carolina@ic.ufrj.br,17/12/2019
13,TRUE,Claudio Miceli de Farias,,Claudio Miceli de Farias,NCE,Adjunto,NAO PREENCHIDO,MIC,2967082,claudiofarias@nce.ufrj.br,None
14,TRUE,Claudson Bornstein,ICP116,Claudson Ferreira Bornstein,PP,Associado,TC,CFB,1124276,cfb@ic.ufrj.br,12/08/1993
15,TRUE,Daniel Alfaro,ICP231,Daniel Gregório Alfaro Vigo,PP,Associado,CC,DGV,1705374,dgalfaro@ic.ufrj.br,10/06/2009


In [45]:
categorias = ["PS", "EX", "AP"]
professores_permanentes = df.loc[(df["Alocar"] == "TRUE") & (~df["Categoria"].isin(categorias))].filter(["Nome curto", "Disciplinas aptas", "Área de conhecimento"])
professores_permanentes.rename(columns={"Nome curto": "professor", "Disciplinas aptas": "qualified_courses", "Área de conhecimento": "expertise"}, inplace=True)

# index(["nome", "disciplinas", "areas"])
professores_permanentes.set_index("professor", inplace=True)
professores_permanentes

,qualified_courses,expertise
professor,,
Adriana Vivacqua,"ICP145,ICP616","ED,ES,H"
Aloisio Pina,,"CD,TC"
Amaury Cruz,,CC
Anamaria Moreira,ICP133,"ES,TC"
Angela Gonçalves,,CC
Carla Delgado,ICP472,"CD,TC,H"
Carolina Marcelino,,CD
Claudio Miceli de Farias,,NAO PREENCHIDO
Claudson Bornstein,ICP116,TC


In [46]:
professores_permanentes = professores_permanentes.to_dict('index')



In [47]:
professores_permanentes["Gabriel Pereira"]

{'qualified_courses': 'ICP143,ICP246', 'expertise': 'SCC'}

In [48]:
for professor in professores_permanentes:
    professores_permanentes[professor]["qualified_courses"] = professores_permanentes[professor]["qualified_courses"].split(",")
    professores_permanentes[professor]["expertise"] = professores_permanentes[professor]["expertise"].split(",")

professores_permanentes


{'Adriana Vivacqua': {'qualified_courses': ['ICP145', 'ICP616'],
  'expertise': ['ED', 'ES', 'H']},
 'Aloisio Pina': {'qualified_courses': [''], 'expertise': ['CD', 'TC']},
 'Amaury Cruz': {'qualified_courses': [''], 'expertise': ['CC']},
 'Anamaria Moreira': {'qualified_courses': ['ICP133'],
  'expertise': ['ES', 'TC']},
 'Angela Gonçalves': {'qualified_courses': [''], 'expertise': ['CC']},
 'Carla Delgado': {'qualified_courses': ['ICP472'],
  'expertise': ['CD', 'TC', 'H']},
 'Carolina Marcelino': {'qualified_courses': [''], 'expertise': ['CD']},
 'Claudio Miceli de Farias': {'qualified_courses': [''],
  'expertise': ['NAO PREENCHIDO']},
 'Claudson Bornstein': {'qualified_courses': ['ICP116'], 'expertise': ['TC']},
 'Daniel Alfaro': {'qualified_courses': ['ICP231'], 'expertise': ['CC']},
 'Daniel Bastos': {'qualified_courses': ['ICP134'], 'expertise': ['TC']},
 'Daniel Sadoc': {'qualified_courses': ['ICP133',
   'ICP362',
   'ICP473',
   'ICP103',
   'ICP350'],
  'expertise': ['CD', 

In [53]:
professores_permanentes.keys()

dict_keys(['Adriana Vivacqua', 'Aloisio Pina', 'Amaury Cruz', 'Anamaria Moreira', 'Angela Gonçalves', 'Carla Delgado', 'Carolina Marcelino', 'Claudio Miceli de Farias', 'Claudson Bornstein', 'Daniel Alfaro', 'Daniel Bastos', 'Daniel Sadoc', 'Daniel Serrão Schneider', 'Eldânae Teixeira', 'Élton Carneiro Marinho', 'Gabriel Pereira', 'Geraldo Bonorino Xexéo', 'Geraldo Zimbrão da Silva', 'Giseli Lopes', 'Hugo Musso', 'Hugo Nobrega', 'João Carlos Pereira', 'João Paixão', 'Jonice de Oliveira', 'Josefino Cabral', 'Juliana França', 'Juliana Valério', 'Luziane Mendonça', 'Marcello Goulart', 'Marcia Helena Costa Fampa', 'Marcia Rosana Cerioli', 'Maria Helena Jardim', 'Maria Luiza Campos', 'Mauro Rincon', 'Mitre Dourado', 'Moacyr Henrique Cruz de Azevedo', 'Mônica Ferreira da Silva', 'Rafael Mello', 'Severino Collier', 'Silvana Rossetto', 'Sulamita Klein', 'Valéria Bastos', 'Vinícius Gusmão', 'Vivian Santos'])